### Imports

In [7]:
import pandas as pd

In [8]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [9]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')
productos = pd.read_csv('../Datasets/tb_productos_descripcion.txt', sep='\t')
productos = productos.drop_duplicates()

stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')
productos_predecir = pd.read_csv('../Datasets/productos_a_predecir.csv')

ventas_grouped = ventas.groupby(['periodo', 'customer_id', 'product_id']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum', # sirve de algo para el modelo? En zulip se me menciona que es la cantidad de solicitudes que hace el cliente, por lo que creo que no
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

ventas_producto = pd.merge(ventas_grouped, productos, on=['product_id'], how='inner')
#final_dataset = pd.merge(ventas_producto, stock, on=['product_id', 'periodo'], how='left')
#final_dataset = pd.merge(ventas_producto, productos_predecir, on=['product_id'], how='inner')
ventas_producto['plan_precios_cuidados'] = ventas_producto['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)
final_dataset = ventas_producto

#final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

In [10]:
final_dataset['periodo'] = final_dataset['periodo'].astype(str)
def determine_quarter(periodo):
    month = periodo[-2:]  # Extrae los últimos dos caracteres que representan el mes
    if month in ['01', '02', '03']:
        return 'Q1'
    elif month in ['04', '05', '06']:
        return 'Q2'
    elif month in ['07', '08', '09']:
        return 'Q3'
    elif month in ['10', '11', '12']:
        return 'Q4'
    else:
        return 'Unknown'  # Para manejar cualquier caso inesperado

# Aplica la función a la columna 'periodo' y crea una nueva columna 'quarter'
final_dataset['quarter'] = final_dataset['periodo'].apply(determine_quarter)
final_dataset['month'] = final_dataset['periodo'].apply(lambda x: x[-2:])
final_dataset.head()

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01


In [11]:
final_dataset.isnull().sum()

periodo                  0
customer_id              0
product_id               0
plan_precios_cuidados    0
cust_request_qty         0
cust_request_tn          0
tn                       0
cat1                     0
cat2                     0
cat3                     0
brand                    0
sku_size                 0
descripcion              0
quarter                  0
month                    0
dtype: int64

Agregamos una columna que indica si el mes es cierre de Q, y otra columna que indique la "edad" del producto, considerando la fecha de la primer venta en el dataset

In [12]:
final_dataset['periodo_temp'] = final_dataset['periodo'].astype(str)
final_dataset['close_quarter'] = final_dataset['periodo_temp'].str[-2:].isin(['03', '06', '09', '12']).astype(int)

final_dataset['first_period'] = final_dataset.groupby('product_id')['periodo'].transform('min')
final_dataset['first_period'] = final_dataset['first_period'].astype(str)

final_dataset['age'] = ((final_dataset['periodo_temp'].str[:4].astype(int) - final_dataset['first_period'].str[:4].astype(int)) * 12 +
                        (final_dataset['periodo_temp'].str[-2:].astype(int) - final_dataset['first_period'].str[-2:].astype(int)))


final_dataset.drop(columns=['periodo_temp', 'first_period'], inplace=True)
display(final_dataset.head())

final_dataset.to_csv('../Datasets/final_dataset_descr.csv', sep='\t', index=False)

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,quarter,month,close_quarter,age
0,201701,10001,20001,0,11,99.43861,99.43861,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01,0,0
1,201701,10002,20001,0,17,38.68301,35.72806,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01,0,0
2,201701,10003,20001,0,17,143.49426,143.49426,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01,0,0
3,201701,10004,20001,0,9,184.72927,184.72927,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01,0,0
4,201701,10005,20001,0,23,19.08407,19.08407,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,Q1,01,0,0


#### Generacion de file para kaggle (para usar en los modelos)

In [13]:
#generar dataset Kaggle
#df.to_csv('../Datasets/df.csv', sep=',', index=False)

#### Formula para calcular el error (para usar en los modelos)

In [14]:
# df_mm['AbsoluteError'] = abs(df_mm['y'] - df_mm['prediction'])
# total_forecast_error = df_mm['AbsoluteError'].sum() / df_mm['y'].sum()